# F1 Generations - Monaco

What if every car to ever compete at the Monaco Grand Prix was put on one grid?

## Whats in this Notebook?
This notebook has the entire process of data collection, parsing, and the race sim. This is for those interested in the details of how the data for the race sim was collected and processed. If you're more interested in just playing with the race simulation, check out the Monaco Generations Race Sim notebook

## Inspiration

Data is fun. F1 is fun. Monaco is... historic. Add it all together, and it's 80% fun!

Shout out to Spanners (of Missed Apex Podcast) for his love of Monaco, saying:
- "You want entertainment, do 30 cars! And ... do Monaco!" - Spanners Feb 6, 2022, Missed Apex Podcast
- "Have a massive grid!" - Spanners Feb 6, 2022, Missed Apex Podcast
- "Bigger grids are much better" - Spanners Feb 6, 2022, Missed Apex Podcast

And to everyone's favorite sim racer dad, Alex Vangeen
- "I'd love to see 30 cars going around Monaco" - Van Gene Feb 6, 2022, Missed Apex Podcast

Additional inspiration from:
- Stuart Taylor "Chain Bear" for helping me fall in love with F1
- AlexDoesF1Stuff for all the amazing F1 data presentation work
- Eric Matthes, author of Python Crash Course for making Python click

# Notes


Using the results where only lead lap cars are considered, are sorted by qualifying times, and cars with no qualifying time are omitted, these are some interesting stats:
- Race is run with 308 cars
- With 108 unique drives
- Lewis Hamilton is the most prolific driver, with 13 entries
- Lined up for the start, the slots would stretch back 1,180 meters, with the last car lined up just behind the Novelle Chicane
- Lewis Hamilton starts on pole, with a time of 1:10.166 set in 2019, by a gap of 0.086s to Valtteri Bottas
- The last car on the grid is Juan Manuel Fangio, winner of the first Monaco GP in 1950, and the only car from that race to be enterted here, as all other cars finished over a lap.
- 2013 is the year with the most entries at 15, likley helped by the two saftey cars and red flag
- All cars from 2013, again due to the saftey cars, perfomed very poorly, running mostly with cars from the 1980s or earlier
- The oldest car in the top 100 is the 2001 Ferrari of Michael Schumacher at 93rd
- The car with the most overtakes is the 1993 McLaren-Ford of Aryton Senna, passing 143 cars to go from 297th to 154th
- The car that dropped back the furthest is the 2011 Mclaren of Lewis Hamilton, dropping 186 positions from 69th to 255th
- Only two cars finished the race where they started, the 2012 Force India of Nico Hulkenburg in 78th and the 2015 Red Bull of Daniil Kvyat in 111th
- There were a total of 7,290 overtakes
- Averaging 93 overtakes per lap
- Or 23 overtakes per car entered


Technical notes:
- Paul Frere is listed in the quali data despite not participating. He was manually added as he raced in the 1955 race, switching with Piero Taruffi in the No 48 Ferrari (I believe on lap 50, finishing 14 laps down). I was unable to find any information on why this happened, and would love to know more. A qualifying entry was added with Taruffi's time so that each Ergas entry has a qualifying entry. However there is not real impact as the car was not on the lead lap.
- Andre Pilette added to quali with Elie Bayol's time, also switched drivers, again no impact as the car was lapped.

# Imports and Local Functions

In [1]:
# Imports
import requests
import json
import pickle
import csv

In [2]:
# Local Functions

# Timing converters
def lap_to_milli(lap):
    "Convert lap time to millisecons"
    try:
        mins = lap.split(':')[0]
        secs = lap.split(':')[1].split('.')[0]
        millis = lap.split(':')[1].split('.')[1]
    except IndexError:
        return(None)

    lap_millis = 0
    lap_millis += int(mins) * 60000
    lap_millis += int(secs) * 1000
    lap_millis += int(millis)
    
    return(lap_millis)

def millis_to_time(millis_total, hrs=True):
    "Converts milliseconds to h:mm:ss.ms"
    millis_total = int(millis_total)
    millis = int(millis_total%1000)
    seconds = str(int((millis_total/1000)%60))
    minutes = str(int((millis_total/(60000))%60))
    if hrs is True:
        hours = str(int((millis_total/(3600000))%24))
    # Two digit checks
    if len(seconds) == 1:
        seconds = f"0{seconds}"
    if len(minutes) == 1:
        minutes = f"0{minutes}"
    if hrs is True:
        end_time = f"{hours}:{minutes}:{seconds}.{millis}"
    else:
        end_time = f"{minutes}:{seconds}.{millis}"
    return(end_time)

# Reading/Writing data helpers
def quick_dict_write(lst, file):
    """Write list of dicts as records in csv"""
    with open(file, 'w', newline='') as f:
        fieldnames = lst[0].keys()
        writer = csv.DictWriter(f, fieldnames=fieldnames, restval='Missing', extrasaction='ignore')
        writer.writeheader()
        for row in lst:
            writer.writerow(row)

def pickle_save(obj, file):
    """Quick pickle save"""
    with open(f"data/saved_objects/{file}.pickle", 'wb') as f:
        pickle.dump(obj, f)
    
def pickle_load(file):
    """Quick pickle load"""
    with open(f"data/saved_objects/{file}.pickle", 'rb') as f:
        obj = pickle.load(f)
    return(obj)

# Get and Parse Race Data

## Ergast API Calls to Get Race Results

API Limits results and asks us to use as few calls as possible to get the data we need. We can focus the data we want to just Monaco GPs, but we will need to run two calls to get all the data.

In [3]:
# Query to get all results for Monaco from 1950-1996
monaco_50_96_response = requests.get('http://ergast.com/api/f1/circuits/monaco/results.json?limit=1070')

In [4]:
# Query to get all results for Monaco from 1996-Current (2021)
monaco_96_21_response = requests.get('http://ergast.com/api/f1/circuits/monaco/results.json?limit=534&offset=1070')

In [5]:
# Convert response to json
monaco_50_96_data = json.loads(monaco_50_96_response.content)
monaco_96_21_data = json.loads(monaco_96_21_response.content)

## Save/Load the Raw Data

Save a copy of the raw json response data so if you make a mistake processing it, you don't need to make another API call for it

In [6]:
# Save raw response data
pickle_save(monaco_50_96_data, 'monaco_50_to_96_raw_data')
pickle_save(monaco_96_21_data, 'monaco_97_to_21_raw_data')

In [4]:
# Load the raw response data (because you have a cat who messed it up)
monaco_50_96_data = pickle_load('monaco_50_to_96_raw_data')
monaco_96_21_data = pickle_load('monaco_97_to_21_raw_data')

## Merge Data Sets

Create single list of all races

In [5]:
race_data = []
race_data.extend(monaco_50_96_data['MRData']['RaceTable']['Races'])
race_data.extend(monaco_96_21_data['MRData']['RaceTable']['Races'])

## Format Results

The Ergast data in it's json structure has a lot of data we don't need and is missing some things we want, so we'll work through each and make our changes, then build a new dictionary and add it to a list of results.

Note, a list of dictionaries may not be the best structure, but it makes sense for me.

Additional note, several fields could (should) be integers and not strings, but it comes as a string, so I kept it like that for consistency and to get the original code setup into this process quicker.

In [70]:
# Set empty results list
results = []

# Build combined list of results by cycling through each race result entry
for race in race_data:
    # Set race data
    season = race['season']
    race_round = race['round']
    # Process each result in the race
    for result in race['Results']:
        # Set number of laps by laps completed by winner
        if result['position'] == '1':
            race_length = result['laps']
        # Set driver name
        driver = f"{result['Driver']['givenName']} {result['Driver']['familyName']}"
        # Build new result dictionary with relevant data fields
        new_result = {
            'season': season,
            'round': race_round,
            'position': result['position'],
            'driver': driver,
            'd_id': result['Driver']['driverId'],
            'constructor': result['Constructor']['name'],
            'driver_nationality': result['Driver']['nationality'],
            'constructor_nationality': result['Constructor']['nationality'],
            'status': result['status'],
            'laps': result['laps'],
            'race_length': race_length,
            'laps_down': str(int(race_length) - int(result['laps']))
        }
        # Check if result has a time associated with it
        if 'Time' in result.keys():
            new_result['milliseconds'] = result['Time']['millis']
            new_result['time'] = result['Time']['time']
            new_result['avg_lap_time'] = str(int(int(new_result['milliseconds'])/int(race_length)))
        else:
            # If no time data, set as blanks
            new_result['milliseconds'] = ''
            new_result['time'] = ''
        
        # Add result to list of results
        results.append(new_result)

In [71]:
# Sample result
results[0]

{'season': '1950',
 'round': '2',
 'position': '1',
 'driver': 'Juan Fangio',
 'd_id': 'fangio',
 'constructor': 'Alfa Romeo',
 'driver_nationality': 'Argentine',
 'constructor_nationality': 'Swiss',
 'status': 'Finished',
 'laps': '100',
 'race_length': '100',
 'laps_down': '0',
 'milliseconds': '11598700',
 'time': '3:13:18.7',
 'avg_lap_time': '115987'}

## Add Identifier

To make each record easily identifiable, we'll add a reference ID that is the driver's name and season ("Lewis Hamilton 2021"). I did this manually, changing some names to remove accents to avoid encoding errors. I don't love that, but I didn't want to spend time on encoding issues.

In [72]:
# Load identifier references
id_ref_file = 'data/driver_id_refs.csv'

ref_data = []

with open(id_ref_file, newline='') as f:
    reader = csv.reader(f)
    headers = []
    # Process file headers to use as keys for dicts
    for header in next(reader):
        headers.append(header.strip().replace(" ", "_"))
    
    for row in reader:
        new = {}
        for idx, entry in enumerate(row):
            new[headers[idx]] = entry.strip()
        ref_data.append(new)

In [73]:
# Sample reference entry
ref_data[0]

{'driver_name': 'Juan Fangio', 'driver_id': 'fangio'}

In [74]:
# Create dict with driver_id keys and standardized reference name as values
ref_dict = {}
for ref in ref_data:
    ref_dict[ref['driver_id']] = ref['driver_name']

# Overwrite the scraped name with the standrd name by looking it up with the driver_id
for record in results:
    record['driver'] = ref_dict[record['d_id']]
    # Set identifier as driver name and season
    record['identifier'] = f"{record['driver']} {record['season']}"

In [75]:
# Sample record
results[0]

{'season': '1950',
 'round': '2',
 'position': '1',
 'driver': 'Juan Fangio',
 'd_id': 'fangio',
 'constructor': 'Alfa Romeo',
 'driver_nationality': 'Argentine',
 'constructor_nationality': 'Swiss',
 'status': 'Finished',
 'laps': '100',
 'race_length': '100',
 'laps_down': '0',
 'milliseconds': '11598700',
 'time': '3:13:18.7',
 'avg_lap_time': '115987',
 'identifier': 'Juan Fangio 1950'}

## Write the Results

Write the results to a csv so you can look at it, because if you're here, you think that's fun too.

In [76]:
# Set the filepath for your new file
out_file = 'data/monaco_50_to_21_race_results.csv'

# Use the handy writer to write each result to it's own row
quick_dict_write(results, out_file)

# Parse Qualifying Data

Qualifying data collected from Wikipedia. I didn't scrape this with Python, but it's very doable. The collected data did require minimal cleaning to ensure columns line up but also specifically to ensure timing formats are consistent as there are a handful of years that use a comma separator for milliseconds instead of a period. 

The qualifying data also required name standardization to match up to the identifiers we setup earlier, in theory, you could program that, but there are so few drivers that needed editing, it's just easier to edit them by hand. A conversion dictionary is included in the data for ease of use.

## Read CSV of Qualifying Data

In [77]:
quali_file = 'data/monaco_quali_data.csv'

quali_data = []

with open(quali_file, newline='') as f:
    reader = csv.reader(f)
    next(reader)
    # For each row, create a new dictionary as a record
    for row in reader:
        new = {}
        new['identifier'] = row[0].strip()
        new['year'] = row[1].strip()
        new['pos'] = row[2].strip()
        new['number'] = row[3].strip()
        new['driver'] = row[4].strip()
        new['constructor'] = row[5].strip()
        # Include raw times as they are in the file
        new['raw_time_1'] = row[6].strip()
        new['raw_time_2'] = row[7].strip()
        new['raw_time_3'] = row[8].strip()
        # Also include converted millis times
        new['time_1'] = lap_to_milli(row[6])
        new['time_2'] = lap_to_milli(row[7])
        new['time_3'] = lap_to_milli(row[8])
        new['quali_time'] = int(row[12])
        # Add new record to quli list
        quali_data.append(new)

## Set Qualifying Time

In [78]:
# Cycle through each quali record
for rec in quali_data:
    # Set empty list of quali times
    q_times = []
    # For any time that has a time, add it to the list of quali times
    if rec['time_1'] is not None:
        q_times.append(rec['time_1'])
    if rec['time_2'] is not None:
        q_times.append(rec['time_2'])
    if rec['time_3'] is not None:
        q_times.append(rec['time_3'])
    # If no quali times, set quali time to zero
    if q_times == []:
        rec['quali_time'] = 0
    # Otherwise, use the fastest time
    else:
        rec['quali_time'] = min(q_times)

## Merge Qualifying with Race Data

Merged using driver identifier, defined as first name last name and season, such as "Lewis Hamilton 2021"

In [79]:
# Build reference list for qualifying data, using driver identifier (First_name Last_name Season)
quali_ref = {}
for rec in quali_data:
    quali_ref[rec['identifier']] = rec['quali_time']

for rec in results:
    # Skip records that didn't participate
    if rec['status'] in ['Did not qualify', 'Did not prequalify', 'Withdrew', 'Excluded']:
        rec['quali_time'] = 0
        continue
    rec['quali_time'] = quali_ref[rec['identifier']]

## Write the Results (Again)

Write the results again, now that we have the qualifying data, just for safekeeping

In [80]:
file = 'data/monaco_data.csv'
quick_dict_write(results, file)

# Race Simulation

Race Qualification  
In order to partake, a car must:
- Have set a qualifying time
- Completed their GP on the lead lap
    - Partially due to space, but mostly because it was prohibitive to get last lap times for pre-96 cars to extrapolate their total time to set their pace.

Race Running  
- Start order set by qualifying time
- Lap pace set at averaged lap time over the race from theri completed GP (Rained? Sorry)
- No pitstops simulated, it's all super mega ultra hards (but if you stopped in your GP, that time is averaged into your race)

## Load Race Data

This is the fun part, so we want to be able to start here.

In [3]:
race_file = 'data/monaco_data.csv'

monaco_data = []

with open(race_file, newline='') as f:
    reader = csv.reader(f)
    headers = []
    # Process file headers to use as keys for dicts
    for header in next(reader):
        headers.append(header.strip().replace(" ", "_"))
    
    for row in reader:
        new = {}
        for idx, entry in enumerate(row):
            new[headers[idx]] = entry.strip()
        monaco_data.append(new)

In [5]:
# Sample record
monaco_data[0]

{'season': '1950',
 'round': '2',
 'position': '1',
 'driver': 'Juan Fangio',
 'd_id': 'fangio',
 'constructor': 'Alfa Romeo',
 'driver_nationality': 'Argentine',
 'constructor_nationality': 'Swiss',
 'status': 'Finished',
 'laps': '100',
 'race_length': '100',
 'laps_down': '0',
 'milliseconds': '11598700',
 'time': '3:13:18.7',
 'avg_lap_time': '115987',
 'identifier': 'Juan Fangio 1950',
 'quali_time': '110200'}

## Race Settings

Variables that are up to interpretation (and are fun to play with)

- laps - Number of laps in the race.
    - Suggested default is 78 as per current GPs (79 in the code to achive this)
- start_gap - Gap between cars for lap one, same for all cars on the grid. Yes it's oversimplified, feel free to change it.
    - Suggested default is 200 (.2s)
- overtake_delta - Pace the overtaking car must have over the car ahead in order to complete an overtake. 
    - Overtaking car must also be able to make up the gap between the cars. 
    - If an overtake is not possible with the gap, but the pace of the behind car is greater than the car ahead, the gap is set to .1s (on their gearbox)
    - Notably, a car with only a .1s pace advantage will never overtake.
    - Suggested default is 1000 (1s)
- overtake_penalty - Time lost when completeing an overtake
    - Suggested default is 500 (.5s)

In [4]:
# Set number of laps
laps = list(range(1, 79))
start_gap = 200
overtake_delta = 1000
overtake_penalty = 0

## Set Start Grid

Grid Qualification
- Set a time in qualifying for their race (Sorry 2018 Max)
- Finished on the lead lap for their race

### Grid for Specific Years

In [76]:
# Years to include
years = ['2021', '2019', '1956', '1950']

# Set blank list for field of cars
field = []

for car in monaco_data:
    # Omit cars with no quali time
    if car['quali_time'] == '0':
        continue
    # Omit cars with no total time (lap down)
    if car['time'] == '':
        continue
    # If car seson in years, add to field
    if car['season'] in years:
        field.append(car)

# Set grid by quali time
grid = sorted(field, key = lambda i: int(i['quali_time']))

# Set initial leader gap as zero
l_gap = 0

# Add additonal race reference data for each car
for idx, car in enumerate(grid, start=1):
    car['current_pos'] = idx
    if idx == 1:
        car['gap'] = 0
    else:
        car['gap'] = start_gap
    car['leader_gap'] = l_gap
    l_gap += start_gap
    car['started'] = idx
    car['lap_time'] = int(car['avg_lap_time'].split('.')[0])
    car['total_time'] = start_gap * (idx-1)

### Grid by Finishing Position for Respective Years

In [23]:
# Years to include
finishing_positions = ['1']

# Set blank list for field of cars
field = []

for car in monaco_data:
    # Omit cars with no quali time
    if car['quali_time'] == '0':
        continue
    # Omit cars with no total time (lap down)
    if car['time'] == '':
        continue
    # If car seson in years, add to field
    if car['position'] in finishing_positions:
        field.append(car)

# Set grid by quali time
grid = sorted(field, key = lambda i: int(i['quali_time']))

# Set initial leader gap as zero
l_gap = 0

# Add additonal race reference data for each car
for idx, car in enumerate(grid, start=1):
    car['current_pos'] = idx
    if idx == 1:
        car['gap'] = 0
    else:
        car['gap'] = start_gap
    car['leader_gap'] = l_gap
    l_gap += start_gap
    car['started'] = idx
    car['lap_time'] = int(car['avg_lap_time'].split('.')[0])
    car['total_time'] = start_gap * (idx-1)

### Mega Grid

That's... why I'm here

In [5]:
# Set blank list for field of cars
field = []

for car in monaco_data:
    # Omit cars with no quali time
    if car['quali_time'] == '0':
        continue
    # Omit cars with no total time (lap down)
    if car['time'] == '':
        continue
    # If car has quli time and finished on lead lap in their GP, add to field
    field.append(car)

# Set grid by quali time
grid = sorted(field, key = lambda i: int(i['quali_time']))

# Set initial leader gap as zero
l_gap = 0

# Add additonal race reference data for each car
for idx, car in enumerate(grid, start=1):
    car['current_pos'] = idx
    if idx == 1:
        car['gap'] = 0
    else:
        car['gap'] = start_gap
    car['leader_gap'] = l_gap
    l_gap += start_gap
    car['started'] = idx
    car['lap_time'] = int(car['avg_lap_time'].split('.')[0])
    car['total_time'] = start_gap * (idx-1)

In [6]:
# Sample entry - Pole for the race
grid[0]

{'season': '2019',
 'round': '6',
 'position': '1',
 'driver': 'Lewis Hamilton',
 'd_id': 'hamilton',
 'constructor': 'Mercedes',
 'driver_nationality': 'British',
 'constructor_nationality': 'German',
 'status': 'Finished',
 'laps': '78',
 'race_length': '78',
 'laps_down': '0',
 'milliseconds': '6208437',
 'time': '1:43:28.437',
 'avg_lap_time': '79595',
 'identifier': 'Lewis Hamilton 2019',
 'quali_time': '70166',
 'current_pos': 1,
 'gap': 0,
 'leader_gap': 0,
 'started': 1,
 'lap_time': 79595,
 'total_time': 0}

## Race Simulation

In [7]:
for lap in laps:
    for car in grid:
         # Set gap
        car['total_time'] += car['lap_time']
        
    # Temporary list for processed cars in new running order
    lap_grid = []
    
    for idx, car in enumerate(grid):
        if car['current_pos'] == 1:
            lap_grid.append(car)
            continue
            
        overtakes = 1
        overtaking = True
        
        while overtaking is True:
            new_gap = car['total_time'] - lap_grid[idx-overtakes]['total_time']
            if car['current_pos'] == 1:
                overtaking = False
                break
            elif new_gap >= 100:
                overtaking = False
            elif new_gap >= 0:
                car['total_time'] = lap_grid[idx-overtakes]['total_time'] + 100
                overtaking = False
            elif abs(new_gap) < overtake_delta:
                car['total_time'] = lap_grid[idx-overtakes]['total_time'] + 100
                overtaking = False
            elif abs(new_gap) >= overtake_delta:
                car['current_pos'] -= 1
                overtakes += 1

        # Add car to lap grid
        if overtakes == 1:
            lap_grid.append(car)
        else:
            lap_grid.insert(car['current_pos']-1, car)
            
    # (Re)order grid by position
    grid = lap_grid[:]
    
    # Update gaps and positions
    leader_gap = 0
    for idx, car in enumerate(grid):
        car['current_pos'] = idx + 1
        if idx == 0:
            car['gap'] = 0
            car['leader_gap'] = 0
        else:
            new_gap = car['total_time'] - grid[idx-1]['total_time']
            leader_gap += new_gap
            car['gap'] = new_gap
            car['leader_gap'] = leader_gap

    # Print timing
    print(f"\nLap {lap} of {len(laps)}")
    print('Driver                      Pos  Interval    To Leader  +/- Total')
    for car in grid:
        driver = car['identifier']
        pos = car['current_pos']
        gap_display = millis_to_time(car['gap'], False)
        leader_gap_display = millis_to_time(car['leader_gap'])
        pos_change = car['started'] - pos
        total_time_display = millis_to_time(car['total_time'])
        
        if pos_change >= 0:
            pos_change = f"+{pos_change}"
        print(f"{driver:26}  {pos:3}  {gap_display:9}   {leader_gap_display:11} {pos_change} {total_time_display}")


Lap 1 of 78
Driver                      Pos  Interval    To Leader  +/- Total
Max Verstappen 2021           1  00:00.0     0:00:00.0   +2 0:01:16.513
Carlos Sainz 2021             2  00:00.315   0:00:00.315 +2 0:01:16.828
Lando Norris 2021             3  00:00.334   0:00:00.649 +2 0:01:17.162
Pierre Gasly 2021             4  00:01.42    0:00:01.691 +4 0:01:18.204
Lewis Hamilton 2019           5  00:01.391   0:00:03.82  -4 0:01:19.595
Valtteri Bottas 2019          6  00:00.240   0:00:03.322 -4 0:01:19.835
Lewis Hamilton 2021           7  00:00.100   0:00:03.422 +5 0:01:19.935
Max Verstappen 2019           8  00:00.731   0:00:04.153 -2 0:01:20.666
Daniel Ricciardo 2018         9  00:00.100   0:00:04.253 -2 0:01:20.766
Sebastian Vettel 2021        10  00:00.100   0:00:04.353 +8 0:01:20.866
Sergio Perez 2021            11  00:00.100   0:00:04.453 +9 0:01:20.966
Sebastian Vettel 2019        12  00:00.262   0:00:04.715 -3 0:01:21.228
Sebastian Vettel 2018        13  00:00.100   0:00:04.815 

Mark Webber 2013            276  00:00.100   0:04:43.469 -72 0:14:52.773
Stirling Moss 1960          277  00:00.100   0:04:43.569 +7 0:14:52.873
Luigi Musso 1958            278  00:00.263   0:04:43.832 +23 0:14:53.136
Nico Hulkenberg 2013        279  00:01.634   0:04:45.466 -71 0:14:54.770
Kimi Raikkonen 2013         280  00:00.100   0:04:45.566 -66 0:14:54.870
Bruce McLaren 1960          281  00:00.813   0:04:46.379 +5 0:14:55.683
Peter Collins 1958          282  00:00.100   0:04:46.479 +18 0:14:55.783
Phil Hill 1960              283  00:02.11    0:04:48.490 +4 0:14:57.794
Giedo van der Garde 2013    284  00:01.38    0:04:49.528 -63 0:14:58.832
Paul di Resta 2013          285  00:00.100   0:04:49.628 -61 0:14:58.932
Esteban Gutierrez 2013      286  00:00.100   0:04:49.728 -55 0:14:59.32
Max Chilton 2013            287  00:02.296   0:04:52.24  -47 0:15:01.328
Jack Brabham 1959           288  00:00.576   0:04:52.600 +2 0:15:01.904
Tony Brooks 1959            289  00:02.439   0:04:55.39 

Ayrton Senna 1987           175  00:12.916   0:05:06.792 +27 0:27:57.226
Nico Rosberg 2016           176  00:06.700   0:05:13.492 -135 0:28:03.926
Sebastian Vettel 2016       177  00:00.100   0:05:13.592 -128 0:28:04.26
Carlos Reutemann 1980       178  00:00.687   0:05:14.279 +19 0:28:04.713
Nelson Piquet 1987          179  00:00.100   0:05:14.379 +37 0:28:04.813
Gilles Villeneuve 1981      180  00:00.100   0:05:14.479 +50 0:28:04.913
Nico Hulkenberg 2016        181  00:01.359   0:05:15.838 -128 0:28:06.272
Sergio Perez 2016           182  00:00.100   0:05:15.938 -124 0:28:06.372
Alain Prost 1988            183  00:00.100   0:05:16.38  +69 0:28:06.472
Nigel Mansell 1986          184  00:00.100   0:05:16.138 +86 0:28:06.572
Graham Hill 1968            185  00:00.100   0:05:16.238 +65 0:28:06.672
Ronnie Peterson 1974        186  00:01.138   0:05:17.376 +43 0:28:07.810
Jody Scheckter 1979         187  00:00.269   0:05:17.645 +39 0:28:08.79
Alan Jones 1981             188  00:00.100   0:05

Heinz-Harald Frentzen 1999  138  00:00.100   0:04:34.84  +26 0:36:17.309
Mika Salo 2000              139  00:00.100   0:04:34.184 +26 0:36:17.409
Jacky Ickx 1971             140  00:00.100   0:04:34.284 +51 0:36:17.509
Denny Hulme 1971            141  00:00.100   0:04:34.384 +62 0:36:17.609
Eddie Irvine 1998           142  00:12.510   0:04:46.894 +25 0:36:30.119
Riccardo Patrese 1992       143  00:00.100   0:04:46.994 +28 0:36:30.219
Ayrton Senna 1991           144  00:16.561   0:05:03.555 +10 0:36:46.780
Ayrton Senna 1990           145  00:00.100   0:05:03.655 +23 0:36:46.880
Mika Salo 1998              146  00:00.100   0:05:03.755 +23 0:36:46.980
Michael Schumacher 1992     147  00:00.100   0:05:03.855 +31 0:36:47.80
Alain Prost 1985            148  00:00.100   0:05:03.955 +22 0:36:47.180
Michele Alboreto 1985       149  00:00.100   0:05:04.55  +26 0:36:47.280
Gerhard Berger 1990         150  00:00.100   0:05:04.155 +27 0:36:47.380
Martin Brundle 1992         151  00:00.100   0:05:04

Piers Courage 1969          166  00:00.100   0:07:15.29  +59 0:47:51.45
Alain Prost 1989            167  00:21.971   0:07:37.0   +27 0:48:13.16
Richard Attwood 1969        168  00:00.100   0:07:37.100 +59 0:48:13.116
Alain Prost 1986            169  00:02.21    0:07:39.121 +51 0:48:15.137
Ayrton Senna 1986           170  00:17.399   0:07:56.520 +31 0:48:32.536
Keke Rosberg 1986           171  00:00.100   0:07:56.620 +39 0:48:32.636
Riccardo Patrese 1982       172  00:13.746   0:08:10.366 +26 0:48:46.382
Ayrton Senna 1987           173  00:21.613   0:08:31.979 +29 0:49:07.995
Nigel Mansell 1986          174  00:00.100   0:08:32.79  +96 0:49:08.95
Daniel Ricciardo 2016       175  00:03.633   0:08:35.712 -136 0:49:11.728
Lewis Hamilton 2016         176  00:00.100   0:08:35.812 -134 0:49:11.828
Carlos Reutemann 1980       177  00:10.259   0:08:46.71  +20 0:49:22.87
Nelson Piquet 1987          178  00:00.100   0:08:46.171 +38 0:49:22.187
Gilles Villeneuve 1981      179  00:00.100   0:08:46.

Mark Webber 2008            234  00:00.100   0:14:02.288 -128 1:07:19.434
Rubens Barrichello 2008     235  00:02.292   0:14:04.580 -122 1:07:21.726
Graham Hill 1965            236  00:00.100   0:14:04.680 +40 1:07:21.826
Kazuki Nakajima 2008        237  00:01.958   0:14:06.638 -119 1:07:23.784
Sebastian Vettel 2008       238  00:00.100   0:14:06.738 -115 1:07:23.884
Lorenzo Bandini 1965        239  00:17.606   0:14:24.344 +39 1:07:41.490
David Coulthard 1996        240  00:32.928   0:14:57.272 -78 1:08:14.418
Olivier Panis 1996          241  00:00.100   0:14:57.372 -68 1:08:14.518
Jackie Stewart 1965         242  00:00.100   0:14:57.472 +35 1:08:14.618
Johnny Herbert 1996         243  00:19.792   0:15:17.264 -71 1:08:34.410
Niki Lauda 1975             244  00:07.890   0:15:25.154 -25 1:08:42.300
Ronnie Peterson 1975        245  00:25.282   0:15:50.436 -11 1:09:07.582
Emerson Fittipaldi 1975     246  00:00.100   0:15:50.536 -10 1:09:07.682
Carlos Pace 1975            247  00:00.100   0:

Fernando Alonso 2013        274  00:00.100   0:25:50.500 -90 1:29:16.550
Jenson Button 2013          275  00:00.100   0:25:50.600 -90 1:29:16.650
Lewis Hamilton 2013         276  00:00.100   0:25:50.700 -90 1:29:16.750
Sebastian Vettel 2013       277  00:00.100   0:25:50.800 -87 1:29:16.850
Nico Rosberg 2013           278  00:00.100   0:25:50.900 -86 1:29:16.950
Valtteri Bottas 2013        279  00:12.781   0:26:03.681 -84 1:29:29.731
Adrian Sutil 2013           280  00:00.100   0:26:03.781 -80 1:29:29.831
Mark Webber 2013            281  00:00.100   0:26:03.881 -77 1:29:29.931
Nico Hulkenberg 2013        282  00:01.913   0:26:05.794 -74 1:29:31.844
Kimi Raikkonen 2013         283  00:00.100   0:26:05.894 -69 1:29:31.944
Jack Brabham 1959           284  00:00.100   0:26:05.994 +6 1:29:32.44
Tony Brooks 1959            285  00:00.100   0:26:06.94  +6 1:29:32.144
Giedo van der Garde 2013    286  00:14.556   0:26:20.650 -65 1:29:46.700
Paul di Resta 2013          287  00:00.100   0:26:20.7

Jack Brabham 1970           153  00:00.100   0:11:23.855 +53 1:27:31.35
Ayrton Senna 1993           154  00:00.100   0:11:23.955 +143 1:27:31.135
Jean Alesi 1991             155  00:38.455   0:12:02.410 +21 1:28:09.590
Nigel Mansell 1991          156  00:00.100   0:12:02.510 +24 1:28:09.690
Jean Alesi 1990             157  00:00.100   0:12:02.610 +25 1:28:09.790
Elio de Angelis 1985        158  00:00.100   0:12:02.710 +23 1:28:09.890
Denny Hulme 1970            159  00:00.100   0:12:02.810 +40 1:28:09.990
Henri Pescarolo 1970        160  00:00.100   0:12:02.910 +51 1:28:10.90
Graham Hill 1969            161  00:19.13    0:12:21.923 +51 1:28:29.103
Damon Hill 1993             162  00:00.100   0:12:22.23  +126 1:28:29.203
Jean Alesi 1993             163  00:00.100   0:12:22.123 +135 1:28:29.303
Ayrton Senna 1989           164  00:39.66    0:13:01.189 +25 1:29:08.369
Jo Siffert 1969             165  00:00.100   0:13:01.289 +52 1:29:08.469
Piers Courage 1969          166  00:00.100   0:13:

Adrian Sutil 2010           104  00:01.490   0:10:29.880 -23 1:39:18.190
Michael Schumacher 2010     105  00:00.100   0:10:29.980 -21 1:39:18.290
Lewis Hamilton 2010         106  00:00.100   0:10:30.80  -21 1:39:18.390
Sebastian Vettel 2015       107  00:00.100   0:10:30.180 -17 1:39:18.490
Sebastien Buemi 2010        108  00:02.693   0:10:32.873 -13 1:39:21.183
Jaime Alguersuari 2010      109  00:01.627   0:10:34.500 -10 1:39:22.810
Daniel Ricciardo 2015       110  00:00.100   0:10:34.600 -10 1:39:22.910
Daniil Kvyat 2015           111  00:00.100   0:10:34.700 +0 1:39:23.10
Kimi Raikkonen 2015         112  00:00.100   0:10:34.800 +3 1:39:23.110
Sergio Perez 2015           113  00:00.100   0:10:34.900 +7 1:39:23.210
Carlos Sainz 2015           114  00:00.100   0:10:35.0   +8 1:39:23.310
Romain Grosjean 2015        115  00:00.100   0:10:35.100 +9 1:39:23.410
Jenson Button 2015          116  00:00.100   0:10:35.200 +12 1:39:23.510
Nico Hulkenberg 2015        117  00:00.100   0:10:35.300 

## Write Race Results

In [8]:
# Set the filepath for your new file
out_file = 'data/mega_grid_results.csv'

# Use the handy writer to write each result to it's own row
quick_dict_write(grid, out_file)

In [9]:
grid[0]

{'season': '2021',
 'round': '5',
 'position': '1',
 'driver': 'Max Verstappen',
 'd_id': 'max_verstappen',
 'constructor': 'Red Bull',
 'driver_nationality': 'Dutch',
 'constructor_nationality': 'Austrian',
 'status': 'Finished',
 'laps': '78',
 'race_length': '78',
 'laps_down': '0',
 'milliseconds': '5936820',
 'time': '1:38:56.820',
 'avg_lap_time': '76113',
 'identifier': 'Max Verstappen 2021',
 'quali_time': '70576',
 'current_pos': 1,
 'gap': 0,
 'leader_gap': 0,
 'started': 3,
 'lap_time': 76113,
 'total_time': 5937214}

In [23]:
y_count = {}

for car in grid:
    if car['season'] in y_count.keys():
        y_count[car['season']] += 1
    else:
        y_count[car['season']] = 1

In [24]:
{k: v for k, v in sorted(y_count.items(), key=lambda item: item[1])}

{'1982': 1,
 '1967': 1,
 '1964': 1,
 '1950': 1,
 '1989': 2,
 '1968': 2,
 '1973': 2,
 '1966': 2,
 '1959': 2,
 '1955': 2,
 '1957': 2,
 '1972': 2,
 '2007': 3,
 '2002': 3,
 '2004': 3,
 '1991': 3,
 '1990': 3,
 '1985': 3,
 '1993': 3,
 '1987': 3,
 '1981': 3,
 '1988': 3,
 '1980': 3,
 '1976': 3,
 '1965': 3,
 '1996': 3,
 '1962': 3,
 '1961': 3,
 '1958': 3,
 '1960': 3,
 '1956': 3,
 '2001': 4,
 '2014': 4,
 '1999': 4,
 '1971': 4,
 '1998': 4,
 '1970': 4,
 '1969': 4,
 '1986': 4,
 '1974': 4,
 '1979': 4,
 '1983': 4,
 '1963': 4,
 '1997': 4,
 '2006': 5,
 '2000': 5,
 '1992': 5,
 '1984': 5,
 '1978': 6,
 '1975': 6,
 '2011': 6,
 '2021': 7,
 '2005': 7,
 '2016': 7,
 '2003': 8,
 '1977': 8,
 '2009': 9,
 '2008': 9,
 '2012': 10,
 '2019': 11,
 '2010': 11,
 '2018': 12,
 '2017': 13,
 '2015': 14,
 '2013': 15}

In [28]:
pos_changes = {}
for car in grid:
    pos = car['current_pos']
    pos_change = car['started'] - pos
    pos_changes[car['identifier']] = pos_change

In [29]:
{k: v for k, v in sorted(pos_changes.items(), key=lambda item: item[1])}

{'Lewis Hamilton 2011': -186,
 'Jenson Button 2011': -179,
 'Sebastian Vettel 2011': -174,
 'Felipe Massa 2008': -161,
 'Heikki Kovalainen 2008': -158,
 'Michael Schumacher 1997': -158,
 'Nico Rosberg 2016': -157,
 'Fernando Alonso 2011': -157,
 'Olivier Panis 1997': -154,
 'Eddie Irvine 1997': -154,
 'Rubens Barrichello 1997': -153,
 'Mark Webber 2011': -152,
 'Sebastian Vettel 2016': -150,
 'Lewis Hamilton 2008': -149,
 'Nico Hulkenberg 2016': -147,
 'Daniel Ricciardo 2016': -146,
 'Kimi Raikkonen 2008': -146,
 'Lewis Hamilton 2016': -144,
 'Kamui Kobayashi 2011': -144,
 'Sergio Perez 2016': -143,
 'Robert Kubica 2008': -138,
 'Jacky Ickx 1972': -131,
 'Mark Webber 2008': -129,
 'Fernando Alonso 2016': -127,
 'Jean-Pierre Beltoise 1972': -124,
 'Rubens Barrichello 2008': -123,
 'Kazuki Nakajima 2008': -119,
 'Alain Prost 1984': -116,
 'Sebastian Vettel 2008': -115,
 'Rene Arnoux 1984': -112,
 'Elio de Angelis 1984': -98,
 'Jean-Eric Vergne 2013': -90,
 'Fernando Alonso 2013': -90,
 '

In [37]:
total_overtakes = 0
for change in pos_changes.values():
    if change > 0:
        total_overtakes += change
        
total_overtakes

7290

In [ ]:
 'Nico Hulkenberg 2012': 0,
 'Daniil Kvyat 2015': 0,

In [36]:
for car in grid:
    if car['identifier'] == 'Lewis Hamilton 2011':
        print(car)

{'season': '2011', 'round': '6', 'position': '6', 'driver': 'Lewis Hamilton', 'd_id': 'hamilton', 'constructor': 'McLaren', 'driver_nationality': 'British', 'constructor_nationality': 'British', 'status': 'Finished', 'laps': '78', 'race_length': '78', 'laps_down': '0', 'milliseconds': '7805583', 'time': '+27.210', 'avg_lap_time': '100071', 'identifier': 'Lewis Hamilton 2011', 'quali_time': '75207', 'current_pos': 255, 'gap': 152138, 'leader_gap': 1881924, 'started': 69, 'lap_time': 100071, 'total_time': 7819138}


In [ ]:
Lewis Hamilton 2011 Mclaren 69th to 255th -186